In [32]:
# Set the seed for reproducibility
import random
import tensorflow as tf
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the dataset
train_data = pd.read_csv('sign_mnist_train.csv')
test_data = pd.read_csv('sign_mnist_test.csv')

# Extract labels and images
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

# Remove 'J' (label 9) and 'Z' (label 25)
valid_labels = [i for i in range(26) if i not in [9, 25]]
train_mask = np.isin(y_train, valid_labels)
test_mask = np.isin(y_test, valid_labels)

X_train = X_train[train_mask]
y_train = y_train[train_mask]
X_test = X_test[test_mask]
y_test = y_test[test_mask]

# Adjust labels to be in range 0-23 instead of 0-24
y_train = [i if i < 9 else i - 1 for i in y_train]
y_test = [i if i < 9 else i - 1 for i in y_test]

# Convert lists to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Reshape images to 32x32 (since the images consists of 32x32 pixels)
X_train = X_train.reshape(-1, 32, 32, 1)
X_test = X_test.reshape(-1, 32, 32, 1)

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Create a fixed validation set and a test set from the testing data
X_val, X_final_test, y_val, y_final_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# One-hot encode labels
y_train_enc = to_categorical(y_train, num_classes=24)
y_val_enc = to_categorical(y_val, num_classes=24)
y_final_test_enc = to_categorical(y_final_test, num_classes=24)


In [34]:
# Let's see how big it is
print(X_train.shape)
print(X_test.shape)
n_total = X_train.shape[0]

(27455, 32, 32, 1)
(7172, 32, 32, 1)


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Define the densely connected model
dense_model = Sequential([
    Flatten(input_shape=(32, 32, 1)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model
dense_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_dense = dense_model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_val, y_val_enc))


Epoch 1/20


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2827 - loss: 2.3754 - val_accuracy: 0.5700 - val_loss: 1.3219
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6973 - loss: 0.9249 - val_accuracy: 0.7100 - val_loss: 0.9362
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8297 - loss: 0.5159 - val_accuracy: 0.7334 - val_loss: 0.9279
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9095 - loss: 0.2890 - val_accuracy: 0.7437 - val_loss: 0.9216
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9410 - loss: 0.1866 - val_accuracy: 0.7474 - val_loss: 1.0296
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9664 - loss: 0.1147 - val_accuracy: 0.7579 - val_loss: 1.0527
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9607 - loss: 0.1248 - val_accuracy: 0.7697 - val_loss: 1.0137
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9728 - loss: 0.0879 - val_accuracy: 0.7948 - val_

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_cnn = cnn_model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_val, y_val_enc))


Epoch 1/20


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5546 - loss: 1.5292 - val_accuracy: 0.8698 - val_loss: 0.4466
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9949 - loss: 0.0373 - val_accuracy: 0.8859 - val_loss: 0.4554
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9984 - loss: 0.0083 - val_accuracy: 0.8751 - val_loss: 0.5138
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.8946 - val_loss: 0.5011
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 5.2028e-04 - val_accuracy: 0.8988 - val_loss: 0.5283
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 2.7991e-04 - val_accuracy: 0.9007 - val_loss: 0.5485
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.7101e-04 - val_accuracy: 0.9013 - val_loss: 0.5693
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.0753e-04 - val_accurac

In [37]:
from sklearn.metrics import confusion_matrix
def evaluate_model(name, model_name, X_final_test, y_final_test_enc, verbose = 2):
    model_eval = model_name.evaluate(X_final_test, y_final_test_enc, verbose = verbose)
    print(f"{name} Model - Test Accuracy: {model_eval[1]}")

    # Detailed evaluation of the model
    y_pred = np.argmax(model_name.predict(X_final_test), axis=1)

    # Compute accuracy per class, skipping index 9 (for J)
    accuracy_per_class = []
    for i in range(24):
        if np.sum(y_final_test == i) > 0:
            accuracy_per_class.append(np.mean(y_pred[y_final_test == i] == i))
        else:
            accuracy_per_class.append(np.nan)  # Handle classes with no samples

    # Filter out NaN values to calculate the median accuracy
    valid_accuracies = [acc for acc in accuracy_per_class if not np.isnan(acc)]
    median_accuracy = np.median(valid_accuracies)

    print(f"Unbiased Median Accuracy: {median_accuracy}")

    # Identify the letter with the highest individual accuracy
    highest_accuracy_class = np.nanargmax(accuracy_per_class)
    print(f"Letter with Highest Accuracy: {chr(highest_accuracy_class + ord('A'))}")

    # Identify the letter with the lowest individual accuracy
    lowest_accuracy_class = np.nanargmin(accuracy_per_class)
    print(f"Letter with Lowest Accuracy: {chr(lowest_accuracy_class + ord('A'))}")

    # Calculate the confusion matrix
  
    conf_matrix = confusion_matrix(y_final_test, y_pred)

    # Set the diagonal elements to zero to exclude correct classifications
    np.fill_diagonal(conf_matrix, 0)

    # Find the indices of the top three errors
    errors = np.unravel_index(np.argsort(-conf_matrix, axis=None), conf_matrix.shape)

    # Get the top three most common errors
    common_errors = [(chr(errors[0][i] + ord('A')), chr(errors[1][i] + ord('A'))) for i in range(3)]
    print(f"Most Common Errors: {common_errors}")

    # Report overall mean accuracy and accuracy per letter
    mean_accuracy = np.nanmean(accuracy_per_class)
    print(f"Overall Mean Accuracy: {mean_accuracy}")

    # Print each letter and its accuracy
    letters = [chr(i + ord('A')) for i in range(26) if i not in [9, 25]]
    for i, acc in enumerate(accuracy_per_class):
        print(f"Letter {letters[i]}: Accuracy {acc}")




In [38]:
evaluate_model("Dense",dense_model, X_val, y_val_enc, 2)

113/113 - 0s - 4ms/step - accuracy: 0.8363 - loss: 0.8788
Dense Model - Test Accuracy: 0.8363078832626343
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Unbiased Median Accuracy: 0.03864824495892457
Letter with Highest Accuracy: B
Letter with Lowest Accuracy: K
Most Common Errors: [('H', 'U'), ('C', 'E'), ('E', 'K')]
Overall Mean Accuracy: 0.04228168717173333
Letter A: Accuracy 0.06707317073170732
Letter B: Accuracy 0.07692307692307693
Letter C: Accuracy 0.019736842105263157
Letter D: Accuracy 0.023255813953488372
Letter E: Accuracy 0.051587301587301584
Letter F: Accuracy 0.038461538461538464
Letter G: Accuracy 0.03125
Letter H: Accuracy 0.07511737089201878
Letter I: Accuracy 0.06802721088435375
Letter K: Accuracy 0.043209876543209874
Letter L: Accuracy 0.009523809523809525
Letter M: Accuracy 0.07
Letter N: Accuracy 0.014184397163120567
Letter O: Accuracy 0.03305785123966942
Letter P: Accuracy 0.02857142857142857
Letter Q: Accuracy 0.02666666666666667
Letter R: Accuracy 0.028169014084507043


In [39]:
evaluate_model("CNN", cnn_model, X_val, y_val_enc, 2)

113/113 - 1s - 5ms/step - accuracy: 0.8957 - loss: 0.8947
CNN Model - Test Accuracy: 0.89570552110672
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Unbiased Median Accuracy: 0.04366376180101671
Letter with Highest Accuracy: B
Letter with Lowest Accuracy: Q
Most Common Errors: [('C', 'E'), ('L', 'H'), ('O', 'E')]
Overall Mean Accuracy: 0.03948595416101395
Letter A: Accuracy 0.04878048780487805
Letter B: Accuracy 0.07692307692307693
Letter C: Accuracy 0.013157894736842105
Letter D: Accuracy 0.023255813953488372
Letter E: Accuracy 0.051587301587301584
Letter F: Accuracy 0.038461538461538464
Letter G: Accuracy 0.0375
Letter H: Accuracy 0.07511737089201878
Letter I: Accuracy 0.047619047619047616
Letter K: Accuracy 0.043209876543209874
Letter L: Accuracy 0.009523809523809525
Letter M: Accuracy 0.06
Letter N: Accuracy 0.04964539007092199
Letter O: Accuracy 0.03305785123966942
Letter P: Accuracy 0.02857142857142857
Letter Q: Accuracy 0.013333333333333334
Letter R: Accuracy 0.0
Letter S: Accuracy 0.

Part 2

In [40]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=10,        # Randomly rotate images by 10 degrees
    width_shift_range=0.1,    # Randomly translate images horizontally by 10% of the width
    height_shift_range=0.1,   # Randomly translate images vertically by 10% of the height
    zoom_range=0.1,           # Randomly zoom images by 10%
    horizontal_flip=True      # Randomly flip images horizontally
)

# Fit the generator to the training data
datagen.fit(X_train)

In [41]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

def create_dense_model_with_regularization(layer_sizes, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 1)))
    for size in layer_sizes:
        model.add(Dense(size, activation='relu', kernel_regularizer=l2(l2_lambda)))
        model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax', kernel_regularizer=l2(l2_lambda)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

def create_cnn_model_with_regularization(conv_layers, dense_layers, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    for filters, kernel_size in conv_layers:
        model.add(Conv2D(filters, kernel_size, activation='relu', padding='same', kernel_regularizer=l2(l2_lambda), input_shape=(32, 32, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Flatten())
    for size in dense_layers:
        model.add(Dense(size, activation='relu', kernel_regularizer=l2(l2_lambda)))
        model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax', kernel_regularizer=l2(l2_lambda)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [64]:
def return_best_model(group_name, model_group, X_train, y_train_enc, batch_size = 30):
    # Train and evaluate each  model
    model_histories = []
    for model in model_group:
        print(f'Model for {group_name}: {model}')
        history = model.fit(
            datagen.flow(X_train, y_train_enc, batch_size=batch_size),
            steps_per_epoch=len(X_train) // 30,
            epochs=50,
            validation_data=(X_val, y_val_enc),
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )
        val_accuracy = history.history['val_accuracy'][-1]
        model_histories.append((val_accuracy, model))

    # Determine the best model based on validation accuracy
    best_val_accuracy, best_model = max(model_histories, key=lambda item: item[0])

    print(f'The best model is: {best_model} with a validation accuracy of: {best_val_accuracy}')
    return model_histories

In [45]:
# Define different dense models to experiment with
dense_models = [
    create_dense_model_with_regularization([512, 256], learning_rate=0.001),
    create_dense_model_with_regularization([1024, 512, 256], learning_rate=0.001),
    create_dense_model_with_regularization([1024, 512, 256, 128], learning_rate=0.001)
]

c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [67]:
# Define different CNN models with regularization to experiment with
cnn_models = [
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3))], [128], learning_rate=0.01, l2_lambda=0.01),
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3)), (128, (3, 3))], [100], learning_rate=0.01, l2_lambda=0.01),
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3)), (128, (3, 3)), (256, (3, 3))], [128], learning_rate=0.01, l2_lambda=0.01)
]




In [61]:
dense_histories = return_best_model("Dense", dense_models, X_train, y_train_enc, 30)

Model for Dense: <Sequential name=sequential_18, built=True>
Epoch 1/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6722 - loss: 1.3625 - val_accuracy: 0.7635 - val_loss: 1.0916 - learning_rate: 1.0000e-05
Epoch 2/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 141us/step - accuracy: 0.6667 - loss: 1.3345 - val_accuracy: 0.7641 - val_loss: 1.0913 - learning_rate: 1.0000e-05
Epoch 3/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6728 - loss: 1.3636 - val_accuracy: 0.7621 - val_loss: 1.0896 - learning_rate: 1.0000e-05
Epoch 4/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 135us/step - accuracy: 0.5000 - loss: 1.6283 - val_accuracy: 0.7621 - val_loss: 1.0894 - learning_rate: 1.0000e-05
Epoch 5/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6755 - loss: 1.3537 - val_accuracy: 0.7716 - val_loss: 1.0881 - learning_rate: 1.0000e-05
Epoch 6/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 155us/step - accuracy: 0.8667 - loss: 1.0984 - val_accuracy: 0.7711 - val_loss: 1.0873 - learning_rate: 1.0000e-

In [68]:
cnn_histories = return_best_model("CNN", dense_models, X_train, y_train_enc, 30)

Model for CNN: <Sequential name=sequential_18, built=True>
Epoch 1/20
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6948 - loss: 1.2979 - val_accuracy: 0.7836 - val_loss: 1.0618 - learning_rate: 1.0000e-05
Epoch 2/20
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 142us/step - accuracy: 0.6000 - loss: 1.7159 - val_accuracy: 0.7844 - val_loss: 1.0616 - learning_rate: 1.0000e-05
Epoch 3/20
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7032 - loss: 1.2888 - val_accuracy: 0.7830 - val_loss: 1.0482 - learning_rate: 1.0000e-05
Epoch 4/20
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 141us/step - accuracy: 0.6667 - loss: 1.3047 - val_accuracy: 0.7842 - val_loss: 1.0474 - learning_rate: 1.0000e-05
Epoch 5/20
915/915 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7030 - loss: 1.2830 - val_accuracy: 0.7962 - val_loss: 1.0308 - learning_rate: 1.0000e-05
Model for CNN: <Sequential name=sequential_19, built=True>
Epoch 1/20
915/915 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6674 - loss: 1.3531 - val_accura

In [69]:
# Combine all histories and models
all_histories = dense_histories + cnn_histories

# Select the best model based on validation accuracy
best_model = max(all_histories, key=lambda x: x[0])[1]


# Evaluate the best model on the final test set
final_eval = best_model.evaluate(X_final_test, y_final_test_enc, verbose=0)
print(f"Best Model: {best_model} - Final Test Accuracy: {final_eval[1]}")


Best Model: <Sequential name=sequential_18, built=True> - Final Test Accuracy: 0.7824874520301819


In [70]:
evaluate_model("CNN", best_model, X_final_test, y_final_test_enc)

113/113 - 0s - 1ms/step - accuracy: 0.7825 - loss: 1.0718
CNN Model - Test Accuracy: 0.7824874520301819
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step
Unbiased Median Accuracy: 0.8240384615384615
Letter with Highest Accuracy: P
Letter with Lowest Accuracy: Q
Most Common Errors: [('E', 'L'), ('L', 'M'), ('I', 'A')]
Overall Mean Accuracy: 0.7812272658392887
Letter A: Accuracy 0.7926829268292683
Letter B: Accuracy 0.8778280542986425
Letter C: Accuracy 0.8618421052631579
Letter D: Accuracy 0.6744186046511628
Letter E: Accuracy 0.6309523809523809
Letter F: Accuracy 0.823076923076923
Letter G: Accuracy 0.825
Letter H: Accuracy 0.9342723004694836
Letter I: Accuracy 0.6258503401360545
Letter K: Accuracy 0.6728395061728395
Letter L: Accuracy 0.9809523809523809
Letter M: Accuracy 0.575
Letter N: Accuracy 0.9645390070921985
Letter O: Accuracy 0.9008264462809917
Letter P: Accuracy 0.9714285714285714
Letter Q: Accuracy 0.9866666666666667
Letter R: Accuracy 0.352112676056338
Letter S: Accuracy 0.5677966